In [1]:
import requests
import time
import asyncio
import aiohttp
import multiprocessing
import concurrent
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import math

## Async vs Sequential

In [ ]:
#https://stackoverflow.com/questions/57126286/fastest-parallel-requests-in-python
async def get(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.read()
            #print("Successfully got url {} with resp of length {}.".format(url, len(resp)))
    except Exception as e:
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def main(urls):
    async with aiohttp.ClientSession() as session:
        #print('t')
        ret = await asyncio.gather(*(get(url, session) for url in urls))
    
    #print("Finalized all. Return is a list of len {} outputs.".format(len(ret)))
def seq_main(urls):
    with requests.Session() as session:
        return [session.get(url) for url in urls]
    
    
test_websites = ['https://google.com'] * 100
start = time.time()
await main(test_websites)
end = time.time()
print(end - start)

# About 30 seconds on my machine - Alvin
# start = time.time()
# seq_main(test_websites)
# end = time.time()
# print(end - start)

t
0.7505519390106201


## Different implementations of Multiprocessing
You can see that we get I/O Blocked

In [ ]:
#https://stackoverflow.com/questions/57126286/fastest-parallel-requests-in-python
test_websites = ['https://google.com'] * 500
start = time.time()
await main(test_websites)
end = time.time()
print(end - start)


n_threads = 16

start = time.time()
with ProcessPoolExecutor(max_workers=n_threads) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(requests.get, web): web for web in test_websites}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            pass
            #print('%r page is %d bytes' % (url, len(data.content)))
end = time.time()
print(end - start)

# https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor
start = time.time()

with ThreadPoolExecutor(max_workers=n_threads) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(requests.get, web): web for web in test_websites}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            pass
            #print('%r page is %d bytes' % (url, len(data.content)))
end = time.time()
print(end - start)

1.6224572658538818
23.54202365875244
21.242860794067383


## Combine Multiprocessing and Asynio: Asyncio across multiple processes

In [ ]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# if u dont impriot it u get an error
# https://stackoverflow.com/questions/62488423/brokenprocesspool-while-running-code-in-jupyter-notebook
# https://stackoverflow.com/questions/78352063/python-concurrent-futures-processpoolexecutor-causes-brokenprocesspool-error-whe
from broke import async_process_main

In [40]:
test_websites = ['https://google.com'] * 100
start = time.time()
await main(test_websites)
end = time.time()
print(end - start)

async def main(urls):
    print('t')
    async with aiohttp.ClientSession() as session:
        
        ret = await asyncio.gather(*(get(url, session) for url in urls))

start = time.time()

n_threads = 16
    # Each process must run a asyncio function?
await async_process_main(test_websites, n_threads)


end = time.time()
print(end - start)


t
0.7827343940734863
7 <Response [200]>
2.8510491847991943


## Use multiple threads to send the custom packets

In [7]:
import time
from scapy.all import IP, TCP, send
from random import randint
from concurrent.futures import ThreadPoolExecutor


def rand_int():
    """Generate a random integer for port or sequence."""
    return randint(1000, 9000)


def send_single_packet(src_ip, dst_ip, dst_port):
    """Send a single custom TCP packet using Scapy."""
    ip_packet = IP(src=src_ip, dst=dst_ip)
    tcp_packet = TCP(
        sport=rand_int(), dport=dst_port, flags="S", seq=rand_int(), window=rand_int()
    )
    send(ip_packet / tcp_packet, verbose=0)


def send_custom_tcp_packets_parallel(src_ip, dst_ip, dst_port, count, n_threads):
    """Send custom TCP packets in parallel using threads."""
    print(f"Sending {count} custom TCP packets to {dst_ip}:{dst_port} using {n_threads} threads...")
    start = time.time()
    
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        futures = [executor.submit(send_single_packet, src_ip, dst_ip, dst_port) for _ in range(count)]
        # Wait for all futures to complete
        for future in futures:
            future.result()  # This prints any exceptions that show up

    print(f"Sent {count} packets in {round(time.time() - start, 2)} seconds")


if __name__ == "__main__":
    # Custom TCP packet sending
    source_ip = "192.168.1.42"
    target_ip = "192.168.1.203"
    target_port = 80
    packet_count = 500
    n_threads = 16

    send_custom_tcp_packets_parallel(source_ip, target_ip, target_port, packet_count, n_threads)
    send_custom_tcp_packets_parallel(source_ip, target_ip, target_port, packet_count, 1)

Sending 500 custom TCP packets to 192.168.1.203:80 using 16 threads...
Sent 500 packets in 3.0 seconds
Sending 500 custom TCP packets to 192.168.1.203:80 using 1 threads...
Sent 500 packets in 30.74 seconds
